In [27]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.optimize import minimize as minimize
from itertools import islice

In [7]:
def data_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [43]:
data = pd.read_excel(r'C:\Users\RE-Giorgio\Downloads\dataPoints.xlsx')
X = np.array(data.iloc[:,:2])
y = np.array(data.iloc[:, 2])
X_train, X_test, X_val, y_train, y_test, y_val = data_split(X, y)

In [60]:
class Mlp:
    
    def __init__(self, X, y, N, sigma):
        
        self.X = X
        self.y = y
        self.N = N
        self.bias = np.zeros(self.N)
        self.weights1 = np.random.normal(0,1,(self.X.shape[1], self.N))
        self.weights2 = np.random.normal(0,1,(self.N, 1))
        self.output = np.zeros(y.shape[0])
        self.rho = np.random.normal(10e-4,10e-5,1)
        self.sigma = sigma
    
    def concatenate(self,l):
        
        l = [array.reshape(-1,1) for array in l]
        return np.concatenate(l)
   
    def act_fun(self,x):
        return (1-np.exp(-2*x*self.sigma))/(1+np.exp(-2*x*self.sigma))

    def forward_propagation(self):
        
        self.hidden_layer = self.act_fun(np.dot(self.X, self.weights1))
        self.output = np.dot(self.hidden_layer, self.weights2)
        return self.output
    
    def loss(self,params, fixed_params):

        seclist = [self.X.shape[1]*self.N, self.N, self.N]
        weights1, weights2, bias = [list(islice(iter(params), 0, i)) for i in seclist]
        y, rho, sigma = fixed_params
        return 0.5 * np.mean(np.square((self.act_fun(self.forward_propagation()) - y))) +\
            rho*np.square(np.linalg.norm(np.hstack((weights1, weights2, bias))))
                          
    def optimization(self):
        function_args = np.array([self.y, self.rho, self.sigma])
        inits = self.concatenate([self.weights1, self.weights2, self.bias])
        return minimize(self.loss, x0 = inits, method='CG', args = function_args)
        

In [61]:
nn = Mlp(X_train, y_train, 10, 1)
nn.optimization()

     fun: 1.4907237258240975
     jac: array([ 3.12924385e-07,  1.49011612e-08, -1.04308128e-07, -1.93715096e-07,
       -1.04308128e-07, -2.83122063e-07,  1.63912773e-07,  4.47034836e-08,
       -2.83122063e-07, -7.45058060e-08, -7.45058060e-08, -1.49011612e-08,
        1.04308128e-07, -4.47034836e-08, -2.98023224e-08,  1.78813934e-07,
        1.78813934e-07,  1.19209290e-07, -2.23517418e-07,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 882
     nit: 4
    njev: 21
  status: 0
 success: True
       x: array([ 4.46098943e-05,  2.56053642e-06, -1.53149349e-05, -2.66684352e-05,
       -1